In [1]:
import json
from parsivar import FindStems

from preprocessor.normalizer import Normalizer
from preprocessor.tokenizer import Tokenizer

# Load Data

In [2]:
def read_json(path):
  file = open(path)
  data = json.load(file)
  return data

In [3]:
PATH = "D:\Data\Docs\\University\\term7\Inforamtion Retrieval\project\search_engine\Dataset\IR_data_news_12k.json"
input_data = read_json(PATH)

In [4]:
print(list(input_data.values())[0])

{'title': 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا', 'content': '\nبه گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان\xa0 قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/\n\n\n', 'tags': ['اعلام زمان', 'قرعه\u200cکشی', 'قرعه\u200cکشی جام', 'قرعه\u200cکشی جام باشگاه\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند'], 'date': '3/15/2022 5:59:27 PM', 'url': 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا', 'category': 'sports'}


In [5]:
print(list(input_data.values())[0].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


In [6]:
contents = [input_data[i]['content'] for i in input_data]

In [7]:
print(len(contents))

12202


In [8]:
print(contents[0])


به گزارش خبرگزاری فارس، کنفدراسیون فوتبال آسیا (AFC) در نامه ای رسمی به فدراسیون فوتبال ایران و باشگاه گیتی پسند زمان  قرعه کشی جام باشگاه های فوتسال آسیا را رسماً اعلام کرد. بر این اساس 25 فروردین ماه 1401 مراسم قرعه کشی جام باشگاه های فوتسال آسیا در مالزی برگزار می شود. باشگاه گیتی پسند بعنوان قهرمان فوتسال ایران در سال 1400 به این مسابقات راه پیدا کرده است. پیش از این گیتی پسند تجربه 3 دوره حضور در جام باشگاه های فوتسال آسیا را داشته که هر سه دوره به فینال مسابقات راه پیدا کرده و یک عنوان قهرمانی و دو مقام دومی بدست آورده است. انتهای پیام/




# Preprocessing

In [9]:
normalizer = Normalizer()
tokenizer = Tokenizer()
stemmer = FindStems()

def preprocess(contents):

  preprocessed_docs = []
  for content in contents:
    # normalizing
    normalized_content = normalizer.normalize(content)
    content_tokens = tokenizer.tokenize(normalized_content)
    tokens = []
    for token in content_tokens:
        token = stemmer.convert_to_stem(token)
          
        tokens.append(token)
    preprocessed_docs.append(tokens)
    # tokens of each doc
  return preprocessed_docs


In [10]:
documents = preprocess(contents)

In [11]:
documents[10]

['به',
 'گزارش',
 'خبرگزاری',
 'فارس',
 'و',
 'به',
 'نقل',
 'از',
 'سایت',
 'باشگاه',
 'تراکتور',
 'میر',
 'معصوم',
 'سهراب',
 'مدیرعامل',
 'این',
 'باشگاه',
 'به',
 'همراه',
 'نمایندگانی',
 'از',
 'استانداری',
 'اذربایجان',
 'شرقی',
 'معاون',
 'عملیات',
 'نیرو',
 'انتظامی',
 'استان',
 'نمایندگانی',
 'از',
 'یگان',
 'ویژه',
 'استان',
 'مدیرکل',
 'ورزش',
 'و',
 'جوان',
 'استان',
 'نماینده',
 'اداره',
 'ورزش',
 'و',
 'جوان',
 'شهرستان',
 'تبریز',
 'و',
 'مدیر',
 'ورزشگاه',
 'یادگار',
 'امام',
 'امروز',
 'سه',
 'شنبه',
 '۲۴',
 'اسفند',
 'از',
 'بخش',
 'مختلف',
 'ورزشگاه',
 'یادگار',
 'امام',
 'ره',
 'تبریز',
 'بازدید',
 'کرده',
 'و',
 'اخرین',
 'وضعیت',
 'و',
 'نواقصات',
 'این',
 'مجموعه',
 'برای',
 'میزبانی',
 'از',
 'هوادار',
 'تراکتور',
 'را',
 'بررسی',
 'کرد&کن',
 'طی',
 'این',
 'بازدید',
 'تمهیدات',
 'لازم',
 'در',
 'راستا',
 'اماده\u200cسازی',
 'این',
 'ورزشگاه',
 'برای',
 'میزبانی',
 'از',
 'بازی',
 'تیم',
 'تراکتور',
 'اندیشیده',
 'شد',
 'انتهای',
 'پیام']

In [12]:
print(len(documents))

12202


# Positional Inverted Indexing

In [13]:
import heapq
from invertedIndex.pinverted_index import Term

In [14]:
def positional_inverted_indexing(documents):
    dictionary = {} 
    for doc_id in range(len(documents)):
        for position in range(len(documents[doc_id])):
            token = documents[doc_id][position]
            if token not in dictionary:
                term = Term()
            else:
                term = dictionary[token]
                
            term.update_posting(doc_id, position)
            dictionary[token] = term

    return dictionary

In [15]:
def delete_the_k_most_repeated_terms(pos_inv_index, k=50):
    max_heap = []
    for term in pos_inv_index.keys():
        heapq.heappush(max_heap, (-pos_inv_index[term].collect_freq, term))

    k_most_repeated = []
    for _ in range(k):
        item = heapq.heappop(max_heap)
        pos_inv_index.pop(item[1])
        k_most_repeated.append(item)
    
    k_most_repeated = [(-key, value) for key, value in k_most_repeated]
    
    return pos_inv_index, k_most_repeated


#### creating positional inverted index 
Here I'm creating a positional inverted index and showing an example

In [16]:
pos_inv_index = positional_inverted_indexing(documents=documents)
pos_inv_index, k_most_repeated = delete_the_k_most_repeated_terms(pos_inv_index)

In [17]:
k_most_repeated

[(219070, 'و'),
 (164285, 'در'),
 (133171, 'به'),
 (92913, 'از'),
 (82972, 'این'),
 (76219, 'که'),
 (68983, 'با'),
 (66735, 'را'),
 (58187, 'اس'),
 (43015, 'کرد&کن'),
 (31007, 'برای'),
 (30969, 'داشت&دار'),
 (28802, 'شد&شو'),
 (27640, 'تیم'),
 (26283, 'ان'),
 (24777, 'کرد'),
 (23942, 'بود&باش'),
 (22370, 'هم'),
 (21716, 'کشور'),
 (19787, 'شد'),
 (19707, 'ما'),
 (18729, 'یک'),
 (17776, 'بازی'),
 (17254, 'می'),
 (16196, 'باید'),
 (15862, 'تا'),
 (15844, 'های\u200c'),
 (15695, 'بر'),
 (15480, 'وی'),
 (14830, 'شده'),
 (14554, 'خود'),
 (14553, 'داد&ده'),
 (14509, 'مجلس'),
 (14416, 'اسلامی'),
 (14375, 'خواست&خواه'),
 (14038, 'گزارش'),
 (13861, 'فارس'),
 (13584, 'گفت'),
 (13190, 'مردم'),
 (13127, 'پیام'),
 (12772, 'رییس'),
 (12705, 'ایران'),
 (12427, 'خبرگزاری'),
 (12251, 'انتهای'),
 (12227, 'دولت'),
 (12204, 'اما'),
 (12117, 'گرفت&گیر'),
 (11952, 'سال'),
 (11022, 'بازیکن'),
 (10779, 'داشت&دارد')]

In [18]:
pos_inv_index['ورزش'].position_in_docs

{3: [63],
 10: [35, 41],
 13: [18, 55, 484, 489, 522, 590, 640, 655, 681],
 29: [519],
 56: [28],
 62: [13, 19, 34, 144, 180, 184, 200, 328],
 65: [476],
 70: [216, 260],
 74: [24],
 77: [47],
 85: [58, 507],
 86: [13,
  23,
  28,
  36,
  49,
  57,
  65,
  74,
  76,
  99,
  126,
  167,
  251,
  296,
  326,
  360,
  368,
  386,
  448,
  523,
  542,
  543,
  548,
  574,
  616,
  641,
  752,
  982],
 89: [433],
 95: [65, 140, 162],
 105: [6, 13, 73, 92, 150, 325, 481, 613, 637, 663],
 106: [24, 50, 114, 136, 208, 228, 297, 374, 412, 431, 482, 610],
 112: [135],
 133: [18, 25, 46, 139],
 140: [681, 783, 1244, 1269],
 142: [1065],
 144: [11, 41, 155, 247, 253, 292, 326, 370, 388, 424, 426, 441],
 154: [5,
  64,
  73,
  112,
  146,
  165,
  198,
  216,
  226,
  245,
  279,
  362,
  370,
  464,
  471,
  483,
  568,
  579,
  625,
  693,
  701,
  709,
  818,
  1034,
  1064,
  1079,
  1195,
  1241,
  1297,
  1397,
  1416,
  1422,
  1429,
  1451,
  1560,
  1739,
  1768,
  1801,
  1895,
  1931,
  

For checking correctness of equality of collection frequency and sum of term frequency in all docs 

In [19]:
print(pos_inv_index['ورزش'].collect_freq)
print(sum(pos_inv_index['ورزش'].term_freq_in_docs.values()))

2669
2669


# Documents in Vector Space

In [20]:
import numpy as np
from invertedIndex.vectorspace import calculate_weights_for_all_terms

In [21]:
calculate_weights_for_all_terms(dictionary=pos_inv_index, collection_size=len(contents))

In [22]:
pos_inv_index['ورزش'].weight_in_docs

{3: 1.1264361823279525,
 10: 1.4655272614098875,
 13: 2.2013294716758307,
 29: 1.1264361823279525,
 56: 1.1264361823279525,
 62: 2.143709419573758,
 65: 1.1264361823279525,
 70: 1.4655272614098875,
 74: 1.1264361823279525,
 77: 1.1264361823279525,
 85: 1.4655272614098875,
 86: 2.7565673503783175,
 89: 1.1264361823279525,
 95: 1.6638828270018915,
 105: 2.252872364655905,
 106: 2.342064985165762,
 112: 1.1264361823279525,
 133: 1.8046183404918228,
 140: 1.8046183404918228,
 142: 1.1264361823279525,
 144: 2.342064985165762,
 154: 2.9549229159703216,
 160: 1.6638828270018915,
 181: 1.1264361823279525,
 184: 2.3812222794300673,
 229: 1.1264361823279525,
 284: 1.6638828270018915,
 302: 2.0029739060838265,
 315: 1.8046183404918228,
 319: 2.0783851922144474,
 324: 1.1264361823279525,
 339: 1.91378128557397,
 341: 1.1264361823279525,
 351: 1.1264361823279525,
 367: 1.6638828270018915,
 371: 1.1264361823279525,
 378: 2.2994985829018395,
 397: 1.1264361823279525,
 418: 1.4655272614098875,
 425: 1

In [23]:
pos_inv_index['ورزش'].term_freq_in_docs

{3: 1,
 10: 2,
 13: 9,
 29: 1,
 56: 1,
 62: 8,
 65: 1,
 70: 2,
 74: 1,
 77: 1,
 85: 2,
 86: 28,
 89: 1,
 95: 3,
 105: 10,
 106: 12,
 112: 1,
 133: 4,
 140: 4,
 142: 1,
 144: 12,
 154: 42,
 160: 3,
 181: 1,
 184: 13,
 229: 1,
 284: 3,
 302: 6,
 315: 4,
 319: 7,
 324: 1,
 339: 5,
 341: 1,
 351: 1,
 367: 3,
 371: 1,
 378: 11,
 397: 1,
 418: 2,
 425: 1,
 435: 4,
 449: 9,
 452: 1,
 454: 3,
 456: 2,
 457: 1,
 458: 13,
 459: 1,
 460: 1,
 461: 1,
 465: 11,
 468: 2,
 469: 2,
 472: 12,
 478: 3,
 479: 2,
 495: 1,
 501: 8,
 505: 2,
 508: 1,
 510: 1,
 524: 1,
 525: 1,
 530: 2,
 531: 2,
 532: 1,
 533: 1,
 534: 9,
 535: 17,
 536: 2,
 538: 6,
 542: 1,
 544: 3,
 545: 3,
 561: 2,
 562: 4,
 563: 7,
 568: 4,
 569: 5,
 570: 5,
 576: 3,
 577: 9,
 592: 1,
 616: 1,
 635: 1,
 641: 1,
 655: 5,
 674: 1,
 683: 1,
 695: 6,
 707: 13,
 708: 3,
 709: 2,
 711: 1,
 714: 1,
 715: 1,
 716: 4,
 728: 3,
 730: 3,
 733: 1,
 743: 1,
 754: 1,
 756: 3,
 781: 3,
 786: 1,
 789: 1,
 790: 4,
 797: 2,
 800: 1,
 818: 1,
 819: 9,
 821

In [24]:
pos_inv_index['فوتبال'].weight_in_doc

{0: 0.7369968554144593,
 1: 0.7369968554144593,
 2: 0.5664718399043003,
 3: 1.0451960815306944,
 7: 0.5664718399043003,
 9: 0.5664718399043003,
 12: 0.7369968554144593,
 15: 0.5664718399043003,
 16: 0.5664718399043003,
 23: 0.7369968554144593,
 29: 0.7369968554144593,
 33: 0.9075218709246182,
 35: 0.7369968554144593,
 36: 0.7369968554144593,
 37: 0.7369968554144593,
 40: 0.5664718399043003,
 42: 0.5664718399043003,
 45: 0.5664718399043003,
 48: 0.5664718399043003,
 53: 1.0451960815306944,
 63: 0.5664718399043003,
 64: 0.5664718399043003,
 66: 1.0072726104329548,
 69: 0.7369968554144593,
 74: 0.5664718399043003,
 76: 0.5664718399043003,
 79: 0.5664718399043003,
 80: 0.8367475949227957,
 81: 1.1974893802347184,
 82: 0.5664718399043003,
 84: 0.5664718399043003,
 85: 1.1070233499412911,
 86: 0.7369968554144593,
 87: 0.7369968554144593,
 88: 0.5664718399043003,
 89: 1.1329436798086006,
 90: 0.5664718399043003,
 91: 0.5664718399043003,
 94: 0.5664718399043003,
 95: 0.5664718399043003,
 96: 0

#### Creating champions list for all terms in dictionary

In [25]:
def create_champ_list(dictionary, k):
    for term in dictionary:
        dictionary[term].create_champ_list(k)

In [26]:
create_champ_list(pos_inv_index, k=50)

In [27]:
pos_inv_index['فوتسال'].get_champ_list()

{4671: 4.083446228235666,
 2478: 3.899478925835592,
 1306: 3.7258195942911247,
 2453: 3.7258195942911247,
 4612: 3.7258195942911247,
 4199: 3.658103823542698,
 4949: 3.658103823542698,
 5304: 3.658103823542698,
 5327: 3.5839295889895517,
 3905: 3.501933776820395,
 4065: 3.501933776820395,
 5392: 3.501933776820395,
 5516: 3.501933776820395,
 6405: 3.501933776820395,
 4613: 3.4102702574450836,
 6283: 3.4102702574450836,
 84: 3.3063507300968618,
 1535: 3.3063507300968618,
 1730: 3.3063507300968618,
 2797: 3.3063507300968618,
 4149: 3.3063507300968618,
 4846: 3.3063507300968618,
 4855: 3.3063507300968618,
 797: 3.1863844399743546,
 4562: 3.1863844399743546,
 4814: 3.1863844399743546,
 5073: 3.1863844399743546,
 1594: 3.0444944346727825,
 2715: 3.0444944346727825,
 4243: 3.0444944346727825,
 4703: 3.0444944346727825,
 4868: 3.0444944346727825,
 4942: 3.0444944346727825,
 5036: 3.0444944346727825,
 5106: 3.0444944346727825,
 5144: 3.0444944346727825,
 6147: 3.0444944346727825,
 6648: 3.04449

#### calculating cosine similarity

In [33]:
from invertedIndex.vectorspace import calculate_tf_for_query, calculate_idf

In [34]:
def calculate_docs_norm(dictionary, collection_size):
    lengths = np.zeros(collection_size)
    for term_obj in dictionary.values():
        for doc_id, weight in term_obj.weight_in_doc.items():
            lengths[int(doc_id)] += weight ** 2
    return np.sqrt(lengths)

In [35]:
docs_norms = calculate_docs_norm(pos_inv_index, len(contents))
docs_norms

array([10.53434085,  7.98068347,  7.65672157, ..., 15.78370391,
       18.4344583 ,  8.2223567 ])

In [30]:
def calculate_cosine_similarity(query_tokens, dictionary, collection_size, norms):
    scores = {} # doc_id -> score
    for term in query_tokens:
        tf_query = calculate_tf_for_query(term, query_tokens)
        idf = calculate_idf(dictionary[term], collection_size)
        weight_term_in_query = tf_query * idf
        for doc_id, weight_term_in_doc in dictionary[term].get_champ_list().items():
            if doc_id not in scores:
                scores[doc_id] = 0
            scores[doc_id] += weight_term_in_query * weight_term_in_doc 
      
    for doc_id in scores:
        scores[doc_id] /= norms[int(doc_id)]
    return scores

## Query Answering

In [46]:
def find_sentences_with_token(content, query_tokens):
    sentences = content.split('.')
    sentences_with_tokens = []

    for sentence in sentences:
        normalized_sentence = normalizer.normalize(sentence)
        if any(token in normalized_sentence for token in query_tokens):
            sentences_with_tokens.append(sentence.strip())

    return sentences_with_tokens

In [71]:
def showing_result(socres, query_tokens):
    for doc_id in socres:
        title = input_data[str(doc_id)]['title']
        url = input_data[str(doc_id)]['url']
        sentences_with_token = find_sentences_with_token(input_data[str(doc_id)]['content'], query_tokens)
        
        print(f'*************[{doc_id}]*************', end='\n\n')
        print('Title: ', title, end='\n\n')
        print('url: ', url, end='\n\n')
        for sentence in sentences_with_token:
            print(f'[{sentence}]')
        print()


In [72]:
def search(query):
    query_tokens = preprocess([query])[0]
    k = 5
    scores = calculate_cosine_similarity(query_tokens, pos_inv_index, len(contents), docs_norms)
    sorted_scores = dict(sorted(scores.items(), key=lambda item: item[1], reverse=True)[:k])
    showing_result(sorted_scores, query_tokens)

#### Simple common single query

In [81]:
query = 'فوتسال'
search(query)

*************[5073]*************

Title:  تیم ملی و ناظم الشریعه نامزد برترین های فوتسال جهان

url:  https://www.farsnews.ir/news/14001019000727/تیم-ملی-و-ناظم-الشریعه-نامزد-برترین-های-فوتسال-جهان

[به ‌گزارش خبرنگار ورزش فارس، سایت رسمی فوتسال پلنت که هر ساله برترین های فوتسال جهان را معرفی می کند در سال ۲۰۲۱ هم برترین های فوتسال جهان را معرفی کرد]
[نام سید محمد ناظم الشریعه سرمربی تیم‌ملی فوتسال کشورمان و همچنین تیم ملی فوتسال ایران جزو ۱۰ گزینه نهایی انتخاب برترینهای فوتسال جهان قرار گرفته است]

*************[990]*************

Title:  واکنش ناظم الشریعه به حضور در تیم ملی فوتسال عراق

url:  https://www.farsnews.ir/news/14001210000405/واکنش-ناظم-الشریعه-به-حضور-در-تیم-ملی-فوتسال-عراق

[فوتسال صحبت هایی کردیم]
[سرمربی سابق تیم ملی فوتسال گفت: در این جلسه در مورد نحوه آموزش و پیشرفت فوتسال در عراق حرف هایی زدیم]
[با توجه به اینکه  ایران به عنوان یک برند در فوتسال دنیا شناخته شده است هر دو کشور می توانند در عرصه آموزش، مربیگری و]

*************[0]*************

Title:  اعلام زمان قرعه 

#### Simple common phrase query

In [82]:
query = 'سرمربی فوتسال'
search(query)

*************[5073]*************

Title:  تیم ملی و ناظم الشریعه نامزد برترین های فوتسال جهان

url:  https://www.farsnews.ir/news/14001019000727/تیم-ملی-و-ناظم-الشریعه-نامزد-برترین-های-فوتسال-جهان

[به ‌گزارش خبرنگار ورزش فارس، سایت رسمی فوتسال پلنت که هر ساله برترین های فوتسال جهان را معرفی می کند در سال ۲۰۲۱ هم برترین های فوتسال جهان را معرفی کرد]
[نام سید محمد ناظم الشریعه سرمربی تیم‌ملی فوتسال کشورمان و همچنین تیم ملی فوتسال ایران جزو ۱۰ گزینه نهایی انتخاب برترینهای فوتسال جهان قرار گرفته است]

*************[990]*************

Title:  واکنش ناظم الشریعه به حضور در تیم ملی فوتسال عراق

url:  https://www.farsnews.ir/news/14001210000405/واکنش-ناظم-الشریعه-به-حضور-در-تیم-ملی-فوتسال-عراق

[فوتسال صحبت هایی کردیم]
[سرمربی سابق تیم ملی فوتسال گفت: در این جلسه در مورد نحوه آموزش و پیشرفت فوتسال در عراق حرف هایی زدیم]
[با توجه به اینکه  ایران به عنوان یک برند در فوتسال دنیا شناخته شده است هر دو کشور می توانند در عرصه آموزش، مربیگری و]
[وی در پاسخ به این سوال که به عنوان سرمربی تیم ملی عراق 

#### Difficult rare single query

In [83]:
query = 'شاملو'
search(query)

*************[11434]*************

Title:  انتخاب اعضای شورای مرکزی حزب «نسل نو»/ تعیین دبیرکل دراولین جلسه

url:  https://www.farsnews.ir/news/14000815000431/انتخاب-اعضای-شورای-مرکزی-حزب-نسل-نو-تعیین-دبیرکل-دراولین-جلسه

[این تشکل جدیدالتاسیس سیاسی با شعار «نسل نو؛ امیدوار، مصلح و تمدن ساز» پا به عرصه سیاست گذاشته است و اسامی شورای مرکزی جدید حزب «نسل نو» به شرح زیر است: ۱- سید احسان قاضی زاده هاشمی  ۲- محسن فتحی ۳- مهدی اقراریان ۴- جعفر بندی شربیانی ۵- سوده نجفی ۶- حامد علامتی ۷- مهدی رمضانی ۸- عطاالله اشرفی اصفهانی ۹- محمدرضاکاشفی ۱۰- محسن میرحاجی ۱۱- امیرابراهیم رسولی ۱۲- حامدزمانی ۱۳- محمدنبی آقاتقی ۱۴- حجت الاسلام میثم آسوپا ۱۵- علی حقیقت ۱۶- نوید خاصه باف ۱۷- امیرخوشخو ۱۸ سید امید میرغضنفری ۱۹- محسن اخلاقی ۲۰- امیدعلی پورحسن ۲۱- سید حامد موسوی ۲۲- حسین محمدیان ۲۳- عادله رضا قلی زاده ۲۴- مازیار عظیم زاده ایرانی ۲۵- سید حسن سیدابراهیم ۲۶- رضا داودی ۲۷- امید شنعه ۲۸- سجاد اکبری ۲۹- آلما فرامرزی ۳۰- علیرضا رحمانی انتخابات بازرسین شورای مرکزی  نیز برگزار و «رضا ترابی»، «حامد توحیدی م

#### Difficult rare phrase query

In [90]:
query = 'قرنطینه کرونا'
search(query)

*************[3569]*************

Title:  طارمی رسید، قائدی اردوی تیم ملی را ترک کرد/ تمهیدات شدید در محل اقامت شاگردان اسکوچیچ

url:  https://www.farsnews.ir/news/14001107000075/طارمی-رسید-قائدی-اردوی-تیم-ملی-را-ترک-کرد-تمهیدات-شدید-در-محل-اقامت

[به گزارش خبرنگار ورزشی خبرگزاری فارس، شب گذشته تست نهایی بازیکنان تیم ملی اعلام و تست کرونای مهدی قائدی مثبت اعلام شد]
[این بازیکن بلافاصله اردوی تیم ملی را ترک کرد و به قرنطینه رفت]
[* مهدی طارمی شب گذشته به اردوی تیم ملی اضافه شد و او هم تست کرونا داد و راهی قرنطینه شد]
[* طبق هماهنگی های صورت گرفته، تدابیر شدیدی برای عدم ورود افراد متفرقه به اردوی تیم ملی در نظر گرفته شده تا کرونا بیش از این به تیم ملی نفوذ نکند]

*************[3660]*************

Title:  تست کرونای نوشاد پس از ۱۰ روز قرنطینه منفی شد

url:  https://www.farsnews.ir/news/14001105000890/تست-کرونای-نوشاد-پس-از-۱۰-روز-قرنطینه-منفی-شد

[به گزارش خبرنگار ورزشی خبرگزاری فارس، نوشاد عالمیان ستاره تنیس روی میز کشورمان که در لیگ فرانسه بازی می‌کند، در چند روز گذشته به کرونا مبتلا شد